# Loop 36 Analysis: Critical Assessment

**Current State:**
- Best CV: 0.008194 (exp_032: GP 0.15 + MLP 0.55 + LGBM 0.3)
- Best LB: 0.0877 (exp_030)
- Target: 0.0347
- Gap to target: 2.53x
- Submissions remaining: 5

**Latest experiments (exp_037 & exp_038):**
- Both similarity weighting experiments failed with IDENTICAL CV (0.022076)
- This is 169% WORSE than baseline
- The identical results suggest a bug in the implementation

**Key Question:** What's causing the failure and what should we try next?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Complete submission history
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877},
]

df = pd.DataFrame(submissions)
print('=== Submission History ===')
print(df)
print(f'\nCV-LB Ratio: {df["lb"].mean() / df["cv"].mean():.2f}x')

In [ ]:
# Linear regression to understand CV-LB relationship
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])
print(f'Linear fit: LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'R² = {r_value**2:.4f}')
print(f'\nIntercept: {intercept:.4f}')
print(f'Target LB: 0.0347')

# Predict LB for our best CV models
best_cv = 0.008194  # exp_032
predicted_lb = slope * best_cv + intercept
print(f'\nPredicted LB for exp_032 (CV={best_cv}): {predicted_lb:.4f}')

# What CV would we need to hit target?
target_lb = 0.0347
required_cv = (target_lb - intercept) / slope
print(f'\nTo reach target LB = {target_lb}:')
print(f'  Required CV = ({target_lb} - {intercept:.4f}) / {slope:.2f} = {required_cv:.6f}')

if required_cv < 0:
    print('\n⚠️ With current CV-LB relationship, target is unreachable!')
    print(f'The intercept alone ({intercept:.4f}) is {intercept/target_lb:.2f}x higher than target ({target_lb})')
    print('\nBUT: This means we need to CHANGE the relationship, not just improve CV!')

In [ ]:
# Analyze the similarity weighting failure
print('=== SIMILARITY WEIGHTING FAILURE ANALYSIS ===')
print()
print('exp_037 (Similarity Weighting): CV 0.022076')
print('exp_038 (Inverse Similarity Weighting): CV 0.022076')
print()
print('IDENTICAL RESULTS! This is highly suspicious.')
print()
print('Evaluator identified the bug:')
print('  - Baseline uses WeightedHuberLoss with target weights [1.0, 1.0, 2.0]')
print('  - Similarity experiments dropped target weighting entirely!')
print('  - Loss computation was different:')
print('    Baseline: loss_fn(pred, y_batch) with target weights')
print('    Similarity: huber(pred, y_batch).mean(dim=1) * sample_weights')
print()
print('The missing target weighting caused the degradation, NOT the sample weighting!')

In [ ]:
# What approaches haven't been tried?
print('=== UNEXPLORED APPROACHES ===')
print()
print('1. AGGRESSIVE FEATURE SELECTION')
print('   - Current: 145 features for 656 samples (24 solvents)')
print('   - Try: Top 20-30 features by importance')
print('   - Rationale: Fewer features = less overfitting to training solvents')
print()
print('2. SIMPLER MODEL ARCHITECTURE')
print('   - Current: [32, 16] MLP')
print('   - Try: Linear model or [16] single layer')
print('   - Rationale: Simpler models generalize better')
print()
print('3. STRONGER REGULARIZATION')
print('   - Current: weight_decay=1e-4')
print('   - Try: weight_decay=1e-2 or 1e-3')
print('   - Rationale: Prevent memorization of training solvents')
print()
print('4. FIX THE SIMILARITY WEIGHTING BUG')
print('   - Apply sample weights WHILE keeping target weights [1.0, 1.0, 2.0]')
print('   - Apply to LGBM (which handles sample weights natively)')
print()
print('5. DIFFERENT ENSEMBLE STRATEGY')
print('   - Try: Stacking with meta-learner instead of weighted average')
print('   - Try: Model selection per solvent type')

In [ ]:
# The key insight: intercept is the problem
print('=== KEY INSIGHT: THE INTERCEPT IS THE PROBLEM ===')
print()
print(f'Current CV-LB relationship: LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'Intercept ({intercept:.4f}) > Target ({target_lb})')
print()
print('This means:')
print('  - Even with CV = 0, predicted LB would be 0.0527')
print('  - The intercept represents systematic generalization error')
print('  - We need approaches that REDUCE THE INTERCEPT')
print()
print('What could reduce the intercept?')
print('  1. Simpler models (fewer parameters to overfit)')
print('  2. Fewer features (less opportunity to memorize)')
print('  3. Stronger regularization (prevent memorization)')
print('  4. Domain adaptation (learn solvent-invariant features)')
print()
print('If we can reduce intercept from 0.0527 to ~0.02:')
print(f'  With CV = 0.008194, predicted LB = {slope:.2f} * 0.008194 + 0.02 = {slope * 0.008194 + 0.02:.4f}')
print('  This would be close to target!')

In [ ]:
# Strategic recommendation
print('=== STRATEGIC RECOMMENDATION ===')
print()
print('PRIORITY 1: Aggressive Feature Selection + Simpler Model')
print('  - Get feature importance from LightGBM')
print('  - Select top 25-30 features')
print('  - Train simpler model (Ridge or [16] MLP)')
print('  - Use strong regularization (weight_decay=1e-2)')
print('  - Rationale: Directly attacks the intercept problem')
print()
print('PRIORITY 2: Fix Similarity Weighting Bug')
print('  - Keep target weights [1.0, 1.0, 2.0]')
print('  - Add sample weights on top')
print('  - Apply to LGBM (native sample weight support)')
print('  - Rationale: The idea was sound, implementation was buggy')
print()
print('PRIORITY 3: Submit exp_032 (best CV)')
print('  - CV 0.008194 is our best')
print('  - Predicted LB: 0.0877')
print('  - Verify the CV-LB relationship holds')
print()
print('With 5 submissions remaining, we should:')
print('  1. Try aggressive simplification first (no submission cost)')
print('  2. If CV improves AND approach is fundamentally different, submit')
print('  3. Save submissions for approaches that could change the CV-LB relationship')